# Multi-Model Serving using OCI Data Science Model Groups

This notebook walks through:
- building SKlearn pipelines
- version controling model artefacts with OCI Data Science 
- deploying SKlearn models to the OCI Data Science Model Catalog
- deploying a single model to an OCI Data Science Model Deployment for real-time inference
- creating a homogenous OCI Data Science Model Group 
- deploying the homogenous model group to a single OCI Data Science Model Deployment
- perform live updates to the Model Deployment


### Load Dependencies

In [19]:
import ads
from ads.model import SklearnModel, ModelVersionSet
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import os
import zipfile
import shutil
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from ads.model import ModelVersionSet
from ads.common.model_metadata import UseCaseType
import requests
from oci.signer import Signer
import pandas as pd
ads.set_auth('resource_principal')

In [2]:
# note we need the latest version of OCI

import oci

print(oci.__version__)

2.163.0


### Set Project Variables

In [3]:
project_id = os.environ["PROJECT_OCID"]
compartment_id = os.environ["NB_SESSION_COMPARTMENT_OCID"]
access_log_group_id = "ocid1.loggroup..."
log_id='ocid1.log...'

### Make Data

In [4]:

X, y = make_classification(random_state=0,n_samples=5000)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)

### Create OCI Data Science Model Version Set

NOTE: This is optional but best practice for version control of individual model artefacts. It also makes it easy to fetch Model OCIDs via ADS when connecting in a new session.

In [ ]:
# Create a model version set
mvs = ModelVersionSet(
    name = "model-group-demo-mvs",
    description = "A model version set for the models in our model group")
mvs.with_compartment_id(compartment_id).with_project_id(project_id).create()


### Create Separate SKLearn Model Pipelines and Upload to OCI Data Science Model Catalog

#### Pipeline 1

In [5]:
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe1 = pipe.set_params(svc__C=10).fit(X_train, y_train)

We now need to save this pipeline to the OCI Data Science Model Catalog. We'll use the ADS SDK and add it to our model version set

In [6]:
artifact_dir1 = "/home/datascience/svm1"
sklearn_model1 = SklearnModel(estimator=pipe1, artifact_dir=artifact_dir1)

In [7]:
sklearn_model1.prepare(
    inference_conda_env="generalml_p311_cpu_x86_64_v1",
    X_sample=X_train,
    y_sample=y_train,
    force_overwrite=True
)


  warnings.warn("slug will be deprecated. Provide conda pack path instead.")



algorithm: Pipeline
artifact_dir:
  /home/datascience/svm1:
  - - output_schema.json
    - runtime.yaml
    - test_json_output.json
    - input_schema.json
    - .model-ignore
    - model.joblib
    - score.py
framework: scikit-learn
model_deployment_id: null
model_id: null

In [13]:
model_id1 = sklearn_model1.save(
    display_name="SVM Demo 1",
    compartment_id=compartment_id,
    model_version_set=mvs.id,
    project_id=project_id
)
print(f"Saved Model 1 to Model Catalog")

Start loading model.joblib from model directory /home/datascience/svm1 ...
Model is successfully loaded.
['output_schema.json', 'runtime.yaml', 'test_json_output.json', 'input_schema.json', '.model-ignore', 'model.joblib', 'score.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

Saved Model 1 to Model Catalog


#### Pipeline 2

In [14]:
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe2 = pipe.set_params(svc__C=9).fit(X_train, y_train)

In [15]:
artifact_dir2 = "/home/datascience/svm2"
sklearn_model2 = SklearnModel(estimator=pipe2, artifact_dir=artifact_dir2)

In [16]:
sklearn_model2.prepare(
    inference_conda_env="generalml_p311_cpu_x86_64_v1",
    X_sample=X_train,
    y_sample=y_train,
    force_overwrite=True
)


  warnings.warn("slug will be deprecated. Provide conda pack path instead.")



algorithm: Pipeline
artifact_dir:
  /home/datascience/svm2:
  - - output_schema.json
    - runtime.yaml
    - input_schema.json
    - .model-ignore
    - model.joblib
    - score.py
framework: scikit-learn
model_deployment_id: null
model_id: null

In [17]:
model_id2 = sklearn_model2.save(
    display_name="SVM Demo 2",
    compartment_id=compartment_id,
    model_version_set=mvs.id,
    project_id=project_id
)
print(f"Saved Model 2 to Model Catalog")

Start loading model.joblib from model directory /home/datascience/svm2 ...
Model is successfully loaded.
['output_schema.json', 'runtime.yaml', 'input_schema.json', '.model-ignore', 'model.joblib', 'score.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

Saved Model 2 to Model Catalog


### Deploy Single Model Deployment Endpoint

Here we show how to deploy a single model to a deployment endpoint using the `ADS SDK`. The benefit here is easy of use with a higher level of abstration. The drawback is that you need at least 1 OCPU per model deployment. Using Model Groups lets us be more cost-effective in our inference deployment.

In [ ]:
sklearn_model1.deploy(
        display_name="SVM 1 Single Deployment Example",
        deployment_log_group_id=access_log_group_id,
     
    )

In [20]:

x_sample = pd.DataFrame(X_test).head().to_json()

In [21]:


config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = 'https://modeldeployment.../predict' # note we can get this dynamically from the .deploy() method
body = x_sample 
headers = {} 
requests.post(endpoint, json=body, auth=auth, headers=headers).json()

{'prediction': [1, 1, 1, 0, 1]}

### Prepare Model Group Artefact

This is a manual task where we create a `.zip` file containing a `score.py` and `runtime.yaml` file. For the homogenous deployment this uses a common scoring file to dynamically load the model in the model group and perform inference against it.

NOTE: While you can use the `score.py` generated by ADS as the base for this, there are some minor changes that need to be made for this to work.

Working `score.py` file:

```
# score.py 1.0 generated by ADS 2.11.19 on 20251107_103309
import os
import sys
import json
import pandas as pd
import numpy as np
from functools import lru_cache
from io import BytesIO
import base64
import logging
from joblib import load

model_name = 'model.joblib'


"""
   Inference script. This script is used for prediction by scoring server when schema is known.
"""


@lru_cache(maxsize=10)
def load_model(model_folder):
    """
    Loads model from the serialized format

    Returns
    -------
    model:  a model instance on which predict API can be invoked
    """
    model_file_name = "model.joblib"
    model_path = model_folder + '/' + model_file_name
    print(f"Model path is: {model_path}")
    if not os.path.exists(model_path):
        error = "Model Path doesn't exist : " + model_path
        print(f"Model path does not exists: {model_path}")
        raise Exception(error)

    # TODO: Load the model from the model_dir using the appropriate loader
    # Below is a sample code to load a model file using `cloudpickle` which was serialized using `cloudpickle`
    # from cloudpickle import cloudpickle
    with open(model_path, "rb") as file:
        model = load(file)

    print("Model is successfully loaded")
    return model


@lru_cache(maxsize=1)
def fetch_data_type_from_schema(input_schema_path=os.path.join(os.path.dirname(os.path.realpath(__file__)), "input_schema.json")):
    """
    Returns data type information fetch from input_schema.json.

    Parameters
    ----------
    input_schema_path: path of input schema.

    Returns
    -------
    data_type: data type fetch from input_schema.json.

    """
    data_type = {}
    if os.path.exists(input_schema_path):
        schema = json.load(open(input_schema_path))
        for col in schema['schema']:
            data_type[col['name']] = col['dtype']
    else:
        print("input_schema has to be passed in in order to recover the same data type. pass `X_sample` in `ads.model.framework.sklearn_model.SklearnModel.prepare` function to generate the input_schema. Otherwise, the data type might be changed after serialization/deserialization.")
    return data_type

def deserialize(data, input_schema_path):
    """
    Deserialize json serialization data to data in original type when sent to predict.

    Parameters
    ----------
    data: serialized input data.
    input_schema_path: path of input schema.

    Returns
    -------
    data: deserialized input data.

    """

    if isinstance(data, bytes):
        logging.warning(
            "bytes are passed directly to the model. If the model expects a specific data format, you need to write the conversion logic in `deserialize()` yourself."
        )
        return data

    data_type = data.get('data_type', '') if isinstance(data, dict) else ''
    json_data = data.get('data', data) if isinstance(data, dict) else data

    if "numpy.ndarray" in data_type:
        load_bytes = BytesIO(base64.b64decode(json_data.encode('utf-8')))
        return np.load(load_bytes, allow_pickle=True)
    if "pandas.core.series.Series" in data_type:
        return pd.Series(json_data)
    if "pandas.core.frame.DataFrame" in data_type or isinstance(json_data, str):
        return pd.read_json(json_data, dtype=fetch_data_type_from_schema(input_schema_path))
    if isinstance(json_data, dict):
        return pd.DataFrame.from_dict(json_data)

    return json_data



def pre_inference(data, input_schema_path):
    """
    Preprocess data

    Parameters
    ----------
    data: Data format as expected by the predict API of the core estimator.
    input_schema_path: path of input schema.

    Returns
    -------
    data: Data format after any processing.

    """
    data = deserialize(data, input_schema_path)
    return data

def post_inference(yhat):
    """
    Post-process the model results

    Parameters
    ----------
    yhat: Data format after calling model.predict.

    Returns
    -------
    yhat: Data format after any processing.

    """
    return yhat.tolist()

def predict(data, model, input_schema_path=os.path.join(os.path.dirname(os.path.realpath(__file__)), "input_schema.json")):
    """
    Returns prediction given the model and data to predict

    Parameters
    ----------
    model: Model instance returned by load_model API
    data: Data format as expected by the predict API of the core estimator. For eg. in case of sckit models it could be numpy array/List of list/Pandas DataFrame
    input_schema_path: path of input schema.

    Returns
    -------
    predictions: Output from scoring server
        Format: {'prediction': output from model.predict method}

    """
    input = pre_inference(data, input_schema_path)
    yhat = post_inference(model.predict(input))
    return {'prediction': yhat}
```

Working `runtime.yaml` file:

```
MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.11/1.0/generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_SLUG: generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_TYPE: data_science
    INFERENCE_PYTHON_VERSION: '3.11'
MODEL_PROVENANCE:
  PROJECT_OCID: ''
  TENANCY_OCID: ''
  TRAINING_CODE:
    ARTIFACT_DIRECTORY: ''
  TRAINING_COMPARTMENT_OCID: ''
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: ''
    TRAINING_ENV_SLUG: ''
    TRAINING_ENV_TYPE: ''
    TRAINING_PYTHON_VERSION: ''
  TRAINING_REGION: ''
  TRAINING_RESOURCE_OCID: ''
  USER_OCID: ''
  VM_IMAGE_INTERNAL_ID: ''
```

Our working Zip file is saved as `model_group_artefact.zip`

### Creating the Model Group

As mentioned earlier in the notebook, Model Groups are a new offering on OCI Data Science and the ADS SDK has not added all features yet to it. Given this, we'll use the OCI SDK directly to build our Model Group and Model Group Deployment. The end result is the same, it just requires slightly more boilerplate code.

In [28]:
dsc = oci.data_science.DataScienceClient(config)

In [31]:
create_model_group_response = dsc.create_model_group(
    create_base_model_group_details=oci.data_science.models.CreateModelGroupDetails(
        create_type="CREATE",
        compartment_id=compartment_id,
        project_id=project_id,
        model_group_details=oci.data_science.models.HomogeneousModelGroupDetails(
            type="HOMOGENEOUS"),
        member_model_entries=oci.data_science.models.MemberModelEntries(
            member_model_details=[
                oci.data_science.models.MemberModelDetails(
                    model_id=model_id1,
                    inference_key="svm1"),
                    oci.data_science.models.MemberModelDetails(
                    model_id=model_id2,
                    inference_key="svm2")]),
        display_name="SVM-Model-Group",
        description="Example of creating a Homogenous Model Group on OCI Data Science"))


In [33]:
# Get the data from response
print(create_model_group_response.data.lifecycle_state)

CREATING


In [35]:
model_group_id = create_model_group_response.data.id

### Upload the Model Group Artefact

In [36]:
file_name = "/home/datascience/model_group_artefact.zip"
content_disposition = "attachment;filename={}".format(os.path.basename(file_name))

with open(file_name, "rb") as f:
    response = dsc.create_model_group_artifact(
        model_group_id=model_group_id,
        model_group_artifact=f,
        content_disposition=content_disposition
    )


In [42]:
print(response.status)

204


In [37]:
print(response.headers)

{'Date': 'Mon, 10 Nov 2025 15:47:39 GMT', 'opc-request-id': '0EEBB041CDBD43A5A0E742D4DB9BB01A/D4C91FDCB21207CC1B091269465241BC/8B7FA08D2EB36775E46F93271F4B97E6', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'ETag': 'bd3e734a-0fe6-4694-9c83-b0a22119891d', 'X-Content-Type-Options': 'nosniff, nosniff', 'Vary': 'Origin', 'Content-Type': 'application/json', 'Connection': 'close'}


### Create Model Group Deployment

In [43]:
# Set Compute Shape 
instance_shape_config_details = oci.data_science.models.ModelDeploymentInstanceShapeConfigDetails(
    memory_in_gbs=16,
    ocpus=1
)

# Set Instance Type
instance_configuration = oci.data_science.models.InstanceConfiguration(
    instance_shape_name="VM.Standard.E4.Flex",
    model_deployment_instance_shape_config_details=instance_shape_config_details
)

# Set Scaling Policy
scaling_policy = oci.data_science.models.FixedSizeScalingPolicy(
    policy_type="FIXED_SIZE",
    instance_count=1  # Adjust as needed
)

# Set Instance Config Details
infrastructure_config_details = oci.data_science.models.InstancePoolInfrastructureConfigurationDetails(
    infrastructure_type="INSTANCE_POOL",
    instance_configuration=instance_configuration,
    scaling_policy=scaling_policy
)

# Set Environment Details (NOTE: Change this to use BYOC option)
environment_config_details = oci.data_science.models.DefaultModelDeploymentEnvironmentConfigurationDetails(
    environment_configuration_type="DEFAULT"
)

# Set Model Group Details
model_group_config_details = oci.data_science.models.ModelGroupConfigurationDetails(
    model_group_id=model_group_id
)

# Set Model Group Deployment Details (infrastructure + model group)
model_group_deployment_config_details = oci.data_science.models.ModelGroupDeploymentConfigurationDetails(
    deployment_type="MODEL_GROUP",
    model_group_configuration_details=model_group_config_details,
    infrastructure_configuration_details=infrastructure_config_details,
    environment_configuration_details=environment_config_details
)

# Set logging details
category_log_details = oci.data_science.models.CategoryLogDetails(
    access=oci.data_science.models.LogDetails(
        log_group_id=access_log_group_id,
        log_id=log_id
    ),
    predict=oci.data_science.models.LogDetails(
        log_group_id=access_log_group_id,
        log_id=log_id
    )
)

# Create Model Deployment using above
create_model_deployment_details = oci.data_science.models.CreateModelDeploymentDetails(
    display_name='MMS SDK',
    description='Test',
    compartment_id=compartment_id,
    project_id=project_id,
    model_deployment_configuration_details=model_group_deployment_config_details,
    category_log_details=category_log_details  # or omit entirely if logging not required
)

response = dsc.create_model_deployment(
    create_model_deployment_details=create_model_deployment_details
)


In [47]:
print(response.data.lifecycle_state)

CREATING


In [46]:
model_group_deployment_url = response.data.model_deployment_url

### Inference against Model Group Deployment

#### Score Model 1

In [51]:
endpoint = model_group_deployment_url+'/predict'

In [55]:
body = x_sample # we use same dataset as before
headers = {'Content-Type':'application/json',
          'opc-request-id':'test-id',
          'model-key':'svm1'} # header goes here
requests.post(endpoint, json=body, auth=auth, headers=headers).json()

{'prediction': [1, 1, 1, 0, 1]}

#### Score Model 2

In [56]:
headers = {'Content-Type':'application/json',
          'opc-request-id':'test-id',
          'model-key':'svm2'} # all we change is the model ocid

requests.post(endpoint, json=body, auth=auth, headers=headers).json()

{'prediction': [1, 1, 1, 0, 1]}

### Live Updates to Model Deployments

One of the benefits of using Model Groups for Deployments is that they support live updates on the compute instance. Here we have a new Model Group (we could also use a Model Group Version Set) that we'll provide to update the live deployment. This requires no downtime for the server.

In [57]:
update_model_group_id = 'ocid1.datasciencemodelgroup...' # OCID for the new Model Group ID

In [58]:
update_model_group_configuration_details = oci.data_science.models.UpdateModelGroupConfigurationDetails( model_group_id=update_model_group_id )

In [67]:
model_deployment_configuration_details = oci.data_science.models.UpdateModelGroupDeploymentConfigurationDetails(
 deployment_type="MODEL_GROUP",
 update_type="LIVE",
 model_group_configuration_details=update_model_group_configuration_details
 )

In [71]:
update_model_deployment_details = oci.data_science.models.UpdateModelDeploymentDetails(
 display_name="MMS SDK",
 description="Live model update to deployment",
 model_deployment_configuration_details=model_deployment_configuration_details
 )

In [72]:
response = dsc.update_model_deployment(
 model_deployment_id=response.data.id, # from the model deployment response object above
 update_model_deployment_details=update_model_deployment_details
 ) 
print("Update submitted. Status:", response.status)

Update submitted. Status: 202


### Summary

In this notebook we have shown how we can create, version and upload models to the OCI Data Science Model Catalog. From there we showed how we would traditionally deploy a model to a single deployment, and then how to take a more compute efficient approach with Model Groups and Multi-Model Serving. We are also able to make live updates to Model Deployments when using the Model Group option making ModelOps seamless in production environments.